In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from skimage.color import rgb2lab, lab2rgb
from skimage.transform import resize
from skimage.io import imsave

import tensorflow as tf
import os
import random
from keras.layers import Conv2D,UpSampling2D
from keras import Sequential
from keras.models import Sequential
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import keras 
from keras.callbacks import EarlyStopping,ModelCheckpoint
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array,load_img
from skimage.color import rgb2lab,lab2rgb
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio

In [ ]:
def load_data(path):
    Input,Output,images=[],[],[]
    for i in tqdm(os.listdir(path)):
        path_=os.path.join(path,i)
        image=load_img(path_,target_size=(224,224),color_mode='rgb')
        image=img_to_array(image)
        image=image/255.0
        
        #augmentation
        image = tf.image.stateless_random_brightness(image, 0.5, seed = (1,5))
        image = tf.image.flip_left_right(image)
        image = tf.image.adjust_saturation(image, 3)
        
        lab=rgb2lab(image)
        Input.append(lab[:,:,0])
        Output.append(lab[:,:,1:]/128)
    return np.array(Input),np.array(Output)

In [ ]:
Input,Output=load_data('/kaggle/input/landscape-image-colorization/landscape Images/color')

In [ ]:
Input=np.expand_dims(Input,axis=-1)
print('Input shape is ' , Input.shape)
print('Output shape is ' , Output.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(Input,Output, test_size=0.2, random_state=44, shuffle =True)
print('X_train shape is ' , X_train.shape)
print('X_test shape is ' , X_test.shape)
print('y_train shape is ' , y_train.shape)
print('y_test shape is ' , y_test.shape)

##Model

In [ ]:
input_ = keras.layers.Input(shape=(224,224,1))
# Encoder
x = keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu',strides=2)(input_)
x = keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
x = keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu',strides=2)(x)
x = keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)
x = keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu',strides=2)(x)
x = keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')(x)
x = keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')(x)
encoder = keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)

# Decoder
x = keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')(encoder)
x = keras.layers.UpSampling2D((2, 2))(x)
x = keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
x = keras.layers.UpSampling2D((2, 2))(x)
x = keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu')(x)
x = keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')(x)
# Adjust the output layer for an RGB image (2 channels)
x = keras.layers.Conv2D(2, (3, 3), padding='same', activation=keras.layers.LeakyReLU(alpha=.5))(x)
decoder = keras.layers.UpSampling2D((2, 2))(x)
# Autoencoder model
autoencoder = keras.models.Model(inputs=input_, outputs=decoder)
autoencoder.summary()

In [ ]:
autoencoder.compile(optimizer = 'adam', loss='mse',metrics = ['accuracy'])


In [ ]:
history = autoencoder.fit(x=X_train,y=y_train,validation_split = 0.15,epochs=20, verbose=1)

In [ ]:
print(autoencoder)

In [ ]:
autoencoder.save('model.h5')

In [ ]:
%pip install git+https://github.com/paulgavrikov/visualkeras

In [ ]:
import visualkeras

In [ ]:

visualkeras.layered_view(autoencoder, to_file='/kaggle/working/architecture.png',scale_xy=10, scale_z=20, max_z=150).show() # write and show

In [ ]:
keras.utils.plot_model(autoencoder, to_file="model.png", show_shapes=True)

In [ ]:
import os
# Download all files from the output directory
output_dir = '/kaggle/working'
zip_filename = 'model.zip'
os.system(f'zip -r {zip_filename} {output_dir}')

# Move the zip file to the /kaggle/working directory
os.system(f'mv {zip_filename} /kaggle/working')

In [ ]:
output1 = autoencoder.predict(X_test)
# output1 = output1 * 128
output1.shape

In [ ]:
n = 9
plt.figure(figsize=(12, 5))
for i in range(n):
    # Display original gray images
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(X_test[i],cmap='gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    # Display original color images
    ax = plt.subplot(3, n, i + 1)
    image = np.zeros((224, 224, 3))
    image[:, :, 0] = X_test[i][:,:,0]
    image[:, :, 1:] = y_test[i]* 128
    plt.imshow(lab2rgb(image))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    # Display pred images
    ax = plt.subplot(3, n, i + 1 + n)
    image[:, :, 0] = X_test[i][:,:,0]
    image[:, :, 1:] = output1[i]* 128
    plt.imshow(lab2rgb(image))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
plt.savefig('result_plot.png')
plt.show()

In [ ]:
# Plot the training and validation accuracy and loss
plt.figure(figsize=(12,4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy [%]')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss [%]')
plt.legend()
plt.savefig('result_plotting.png')
plt.show()

## TF DS

In [ ]:
list_ds = tf.data.Dataset.list_files('flower_photos/*/*', shuffle=False)

In [ ]:
iter_list_ds = iter(list_ds)
next(iter_list_ds)

In [ ]:
path = next(iter_list_ds)

In [ ]:
path

In [ ]:
list_ds = list_ds.shuffle(image_count, reshuffle_each_iteration=False)

In [ ]:
val_size = int(image_count * 0.2)
train_ds = list_ds.skip(val_size)
val_ds = list_ds.take(val_size)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
def get_label(file_path):
  # Convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  return tf.argmax(one_hot)

In [ ]:
def decode_img(img):
  # Convert the compressed string to a 3D uint8 tensor
  img = tf.io.decode_jpeg(img, channels=3)
  # Resize the image to the desired size
  return img

In [ ]:
def resize_and_rescale(image):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
  image = (image / 255.0)
  return image

In [ ]:
def augment(image):
  seed = [1234, 1234]
  image = tf.image.resize_with_crop_or_pad(image, IMG_SIZE + 6, IMG_SIZE + 6)
  # Make a new seed.
  new_seed = tf.random.split(seed, num=1)[0, :]
  # Random crop back to the original size.
  image = tf.image.stateless_random_crop(
      image, size=[IMG_SIZE, IMG_SIZE, 3], seed=seed)
  # Random brightness.
  image = tf.image.stateless_random_brightness(
      image, max_delta=0.5, seed=new_seed)
  image = tf.clip_by_value(image, 0, 1)
  return image

In [ ]:
def process_path(file_path):
  label = get_label(file_path)
  # Load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  img = resize_and_rescale(img)
  img = augment(img)
  return img, label

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
train_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [ ]:
for image, label in train_ds.take(3):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

In [ ]:
def configure_for_performance(ds):
  ds = ds.cache()
  ds = ds.shuffle(buffer_size=1000)
  ds = ds.batch(batch_size)
  ds = ds.prefetch(buffer_size=AUTOTUNE)
  return ds

train_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)